In [1]:
import pandas as pd
from footix.models.bayesian import Bayesian
import footix.implied_odds as odds
import collections
import pymc as pm

In [2]:
dataset = pd.read_csv("data/ligue1_2024_2025.csv")

In [3]:
model = Bayesian(n_teams=18, n_goals=10)

In [4]:
model.fit(X_train=dataset)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>BinaryGibbsMetropolis: [grp_att, grp_def]
>NUTS: [home, intercept, sigma_att, sigma_def, attack, defense]


Output()

Sampling 4 chains for 2_000 tune and 3_000 draw iterations (8_000 + 12_000 draws total) took 282 seconds.
There were 2374 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [5]:
fixture = pd.read_csv("matches_info.csv")
# Assuming fixture and odds are already defined DataFrames
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['Home team']:<30} {match['Away team']:<30}")
    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=match["Home team"], away_team=match["Away team"]).return_probas()
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [float(match["H"].replace(',', '.')), float(match["D"].replace(',', '.')), float(match["A"].replace(',', '.'))]
    odds_result = odds.shin(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")

    print("#" * 50)

##################################################
Nice                           Rennes                        
  Probabilities: Home: 0.49, Draw: 0.24, Away: 0.26
  Odds: Home: 0.44, Draw: 0.28, Away: 0.28
##################################################
##################################################
St Etienne                     Reims                         
  Probabilities: Home: 0.27, Draw: 0.25, Away: 0.47
  Odds: Home: 0.27, Draw: 0.28, Away: 0.45
##################################################
##################################################
Lille                          Nantes                        
  Probabilities: Home: 0.50, Draw: 0.25, Away: 0.24
  Odds: Home: 0.62, Draw: 0.22, Away: 0.16
##################################################
##################################################
Lyon                           Montpellier                   
  Probabilities: Home: 0.61, Draw: 0.19, Away: 0.17
  Odds: Home: 0.71, Draw: 0.17, Away: 0.12
###############

In [6]:
#pm.summary(model.trace)

In [7]:
# Assuming fixture and odds are already defined DataFrames
fixture = pd.read_csv("matches_info.csv")
selections = []

for idx, match in fixture.iterrows():
    home_team = match["Home team"]
    away_team = match["Away team"]
    name = f"{home_team} vs {away_team} - "

    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]

    # Display odds from the function
    odds_list = [float(match["H"].replace(',', '.')), float(match["D"].replace(',', '.')), float(match["A"].replace(',', '.'))]
    odds_result = odds.shin(odds=odds_list)[0]
    home_implied = odds_result[0]
    draw_implied = odds_result[1]
    away_implied = odds_result[2]

    home_odd = odds_list[0]
    draw_odd = odds_list[1]
    away_odd = odds_list[2]

    # Calculate the value for each outcome
    home_value = home_prob - home_implied
    draw_value = draw_prob - draw_implied
    away_value = away_prob - away_implied

    # Determine the best selection
    if home_value >= draw_value and home_value >= away_value:
        best_prob = home_prob
        best_odd = home_odd
        sel = "home"
    elif draw_value >= home_value and draw_value >= away_value:
        best_prob = draw_prob
        best_odd = draw_odd
        sel = "draw"
    else:
        best_prob = away_prob
        best_odd = away_odd
        sel = "away"
    # Append the selection to the list
    selections.append({
        "name": name+sel,
        "probability": best_prob,
        "odds_bookie": best_odd
    })

In [8]:
print(selections)

[{'name': 'Nice vs Rennes - home', 'probability': 0.5308400184072326, 'odds_bookie': 2.15}, {'name': 'St Etienne vs Reims - away', 'probability': 0.5191936123310301, 'odds_bookie': 2.1}, {'name': 'Lille vs Nantes - away', 'probability': 0.2654404363665571, 'odds_bookie': 5.5}, {'name': 'Lyon vs Montpellier - away', 'probability': 0.10812917160774263, 'odds_bookie': 6.9}, {'name': 'Lens vs Toulouse - away', 'probability': 0.26428175645717616, 'odds_bookie': 4.0}, {'name': 'Strasbourg vs Auxerre - away', 'probability': 0.28813577706461163, 'odds_bookie': 3.75}, {'name': 'Angers vs Brest - home', 'probability': 0.38155329181583525, 'odds_bookie': 2.95}, {'name': 'Marseille vs Le Havre - away', 'probability': 0.16406536122110965, 'odds_bookie': 9.0}, {'name': 'Nantes vs Monaco - home', 'probability': 0.36497384726418497, 'odds_bookie': 3.75}, {'name': 'Auxerre vs Lille - home', 'probability': 0.435925358639452, 'odds_bookie': 3.5}, {'name': 'Brest vs Lyon - home', 'probability': 0.38507461

In [9]:
from footix.strategy.strategies import realKelly

In [10]:
realKelly(selections[:8], bankroll=30, max_multiple=1, optimizer_kwargs={"popsize":100, "workers": -1})

/Users/shaheenacheche/miniconda3/envs/footix/lib/python3.10/site-packages/scipy/optimize/_differentialevolution.py:488: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


/Users/shaheenacheche/miniconda3/envs/footix/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/Users/shaheenacheche/miniconda3/envs/footix/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)



2025-01-03 17:26:10 - Optimization finished. Runtime --- 3.738 seconds ---

Objective: -3.45149
Certainty Equivalent: 31.547

Nice vs Rennes - home @2.15 - € 3
St Etienne vs Reims - away @2.1 - € 2
Lille vs Nantes - away @5.5 - € 3
Lens vs Toulouse - away @4.0 - € 1
Strasbourg vs Auxerre - away @3.75 - € 1
Angers vs Brest - home @2.95 - € 2
Marseille vs Le Havre - away @9.0 - € 2
Bankroll used 13.41199987685189 €
